In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf
from sklearn import metrics
import pickle

In [2]:
data_dir = "../data/"
pickle_file = 'tf_data.pickle'

with open(data_dir + pickle_file, 'rb') as f:
    save = pickle.load(f)
    train_dataset = np.matrix(save['train_data'], dtype='float32')
    train_labels = np.matrix(save['train_labels'], dtype='float32')
    valid_dataset = np.matrix(save['validate_data'], dtype='float32')
    valid_labels = np.matrix(save['validate_labels'], dtype='float32')
    test_dataset = np.matrix(save['test_data'], dtype='float32')
    test_bids = list(save['test_business_ids'])
    #test_bids = np.ravel(test_bids)
    del save  # hint to help gc free up memory
    print 'Training set', train_dataset.shape, train_labels.shape
    print 'Validation set', valid_dataset.shape, valid_labels.shape
    print 'Test set', test_dataset.shape, len(test_bids)

Training set (1600, 1024) (1600, 9)
Validation set (400, 1024) (400, 9)
Test set (10000, 1024) 10000


In [45]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

#set to -1 to use all data
train_subset = 1600
input_size = 1024
labels_size = 9

#def f1_score(y_true, y_pred):
#    l, w = y_pred.shape
#    y_true_1d = y_true.reshape(l * w, 1)
#    y_pred_1d = y_pred.reshape(l * w, 1)
#    y_int = np.intersect1d(y_true_1d, y_pred_1d)
    
    
#    tp = np.
#    fp =
#    fn =
#    p = tp / (tp + fp)
#    r = tp / (tp + fn)
#    f1 = 2 * ((p * r) / (p + r) )
    
    
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset, :])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    weights = tf.Variable(
        tf.truncated_normal([input_size, labels_size]))
    #biases = tf.Variable(tf.zeros([labels_size]))
    biases = tf.Variable(tf.zeros([labels_size]))

    # Training computation.
    # We multiply the inputs with the weight matrix, and add biases. We compute
    # the softmax and cross-entropy (it's one operation in TensorFlow, because
    # it's very common, and it can be optimized). We take the average of this
    # cross-entropy across all training examples: that's our loss.
    logits = tf.matmul(tf_train_dataset, weights) + biases
    
    loss = tf.reduce_mean(
        #This uses sigmoid cross entropy, which allows for multiple labels
        tf.nn.sigmoid_cross_entropy_with_logits(logits, tf_train_labels))

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.AdamOptimizer(0.1).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.sigmoid(logits)
    valid_prediction = tf.nn.sigmoid(
        tf.matmul(tf_valid_dataset, weights) + biases)
    test_prediction = tf.nn.sigmoid(
        tf.matmul(tf_test_dataset, weights) + biases)

In [46]:
num_steps = 10000

def f1_score(y_true, y_pred):
    l, w = y_pred.shape
    f1 = np.zeros(l, dtype='float32')
    #print f1.shape
    y_pred = np.array(np.array(np.round(y_pred), dtype=bool), dtype='float32')
    #return 0
    #print y_pred[:10]
    #y_true.reshape(9,len(y_pred))
    #print y_pred.shape
    #print y_true.shape
    for i in range(len(y_pred)):
        #print y_pred[i]
        #print np.ravel(y_true[i])
        f1[i]= metrics.f1_score(np.ravel(y_true[i]), y_pred[i], average='binary')
        #if f1[i] < 1.:
            #print y_true[i]
            #print y_pred[i]
    f1_mean = np.mean(f1)
    return f1_mean

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print 'Initialized'
    for step in xrange(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 1000 == 0):
            #print predictions.shape
            print 'Loss at step', step, ':', l
            #print "F1 Score:", (valid_labels)
            print 'Training accuracy: ',f1_score(
               train_labels[:train_subset, :], predictions)
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print 'Validation accuracy:', f1_score(
                 valid_labels, valid_prediction.eval())
            
            test_results = test_prediction.eval()
            
#Currently seems to overfit/ converge very quickly
#Training 99.8%
#Validation 87.5 %

Initialized
Loss at step 0 : 74.3614
Training accuracy:  0.522202
Validation accuracy: 0.655496
Loss at step 1000 : 0.00107559
Training accuracy:  0.998125
Validation accuracy: 0.845191
Loss at step 2000 : 0.00033426
Training accuracy:  0.998125
Validation accuracy: 0.844384
Loss at step 3000 : 0.000166464
Training accuracy:  0.998125
Validation accuracy: 0.84416
Loss at step 4000 : 9.40158e-05
Training accuracy:  0.998125
Validation accuracy: 0.844144
Loss at step 5000 : 5.61269e-05
Training accuracy:  0.998125
Validation accuracy: 0.846049
Loss at step 6000 : 3.44529e-05
Training accuracy:  0.998125
Validation accuracy: 0.847869
Loss at step 7000 : 2.14285e-05
Training accuracy:  0.998125
Validation accuracy: 0.847869
Loss at step 8000 : 1.3411e-05
Training accuracy:  0.998125
Validation accuracy: 0.847769
Loss at step 9000 : 8.41798e-06
Training accuracy:  0.998125
Validation accuracy: 0.847912


/home/fcarey/anaconda2/lib/python2.7/site-packages/sklearn/metrics/classification.py:1076: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


In [ ]:
train_dataset = save['train_dataset']
train_labels = save['train_labels']
valid_dataset = save['valid_dataset']
valid_labels = save['valid_labels']
test_dataset = save['test_dataset']
test_bids = save['test_bids']
del save  # hint to help gc free up memory
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

In [31]:
#print test_bids

test_results = np.array(np.round(test_results), dtype=bool)
output = "business_id,labels\n"

for i in range(len(test_bids)):
    print i
    tags = np.argwhere(test_results[i])
    tags_str = ' '.join('%d'%F for F in tags[:] )
    output += test_bids[i]+ "," + tags_str + "\n"
    
print output
f = open('test_submission', 'w')
f.write(output)
f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [14]:
output

"set(['af0iy', 's4n0d', 's1e3l', 'yinms', '3i26e', 'rl851', '0zro, 0 1 2\n"

In [3]:
test_dataset[1]

matrix([[ 2.91015625,  6.21484375,  1.28222656, ...,  3.83398438,
          8.265625  ,  4.45703125]], dtype=float32)